In [ ]:
from tqdm import tqdm
from array_lib import *
from point3d_lib import Point
from ply_creation_lib import create_ply
from skimage.morphology import skeletonize
import matplotlib.pyplot as plt
import pydicom as dicom
import numpy as np
import itertools
import struct
import pickle
import time
import os

In [ ]:
"""
input_folder - must contain dcm files directly within it and only of one scan.
offsets - range of colors to include
seed - the entry coordinate to the aorta, must be the center.
threshold - normalized image color lower threshold
image_center - point between the arteries of interest
"""
input_folder = f'{os.getcwd()}\\20240923'
offsets = (-15, 20)
seed = (20, 310, 160)
image_center = (0, 280, 230)

In [ ]:
def read_dicom(input_folder: str) -> np.ndarray:
    files: list[str] = os.listdir(input_folder)
    data = [dicom.dcmread(f'{input_folder}\\{file}') for file in files if file.endswith('.dcm')]
    image = np.array([dicom.pixel_array(datum) for datum in data])
    return image

def normalize_image_colors(image: np.ndarray) -> np.ndarray:
    min_val = np.min(image)
    max_val = np.max(image)
    image = (image - min_val) / (max_val - min_val) * 255
    return image

image = normalize_image_colors(read_dicom(input_folder))

In [ ]:
filtered_mask = custom_floodfill_3d(image, seed_point=seed, new_value=-1, offsets=offsets)
eroded_mask = erode_3d(filtered_mask)
heartless_mask = remove_heart(eroded_mask)
trimmed_mask = distinguish_3d(heartless_mask)

In [ ]:
skeleton_mask = skeletonize(trimmed_mask)
closest_skeletons = find_closest_skeletons(skeleton_mask, image_center)
skeleton_points = np.concatenate([i[0] for i in closest_skeletons])
filtered_skeleton_mask = np.zeros_like(skeleton_mask).astype(bool)
filtered_skeleton_mask[skeleton_points[:, 0], skeleton_points[:, 1], skeleton_points[:, 2]] = True

selected_skeletons = floodfill_nearby_skeletons(heartless_mask, closest_skeletons)

In [ ]:
def add_skeleton_points(skeletons: list[np.ndarray], filtered_skeleton_mask: np.ndarray, offsets: list) -> list[Point]:
    """
    Convert the skeleton mask into skeleton structures.
    """
    skeleton: list[Point] = []
    for skeleton_point in skeletons:
        skeleton.append(Point(skeleton_point))

    for point in skeleton:
        point_surround = point.get_surround_points(offsets)
        nearby_points = [p for p in point_surround if filtered_skeleton_mask[p]]
        for p in nearby_points:
            if not filtered_skeleton_mask[p]:
                continue
            for another_point in skeleton:
                if np.array_equal(another_point.coordinates, p):
                    point.add_nearby(another_point)
                    break
        point.check_state()
    return skeleton

skeletons = [s[0] for s in closest_skeletons]

offsets = list(itertools.product([-1, 0, 1], repeat=3))
offsets.remove((0, 0, 0))

left_skeleton = add_skeleton_points(skeletons[0], filtered_skeleton_mask, offsets)
right_skeleton = add_skeleton_points(skeletons[1], filtered_skeleton_mask, offsets)

In [ ]:
def remove_skeleton_close_ends(skeleton: list[Point], closeness: int):
    """
    Remove skeleton ends if they are closer than some 20 points of distance to the nearest cross.
    """    
    removed = True
    while removed:
        removed = False
        skeleton_ends = [p for p in right_skeleton if p.end]
        for end in skeleton_ends:
            if end.is_cross_close(closeness):
                end.remove_point()
                removed = True
    skeleton = [p for p in skeleton if p.value > -1]
    return skeleton

right_skeleton = remove_skeleton_close_ends(right_skeleton, closeness=20)
left_skeleton = remove_skeleton_close_ends(left_skeleton, closeness=20)

In [ ]:
ends = [p for p in right_skeleton if p.end]
longest_path: list = []
for end in ends:
    distance, points = end.distance_to_far_end()
    longest_path.append((distance, points))
longest_path.sort(key = lambda x: x[0])
longest_skeleton_path: list[Point] = longest_path[-1][1]
longest_skeleton_path_coords = np.array([p.coordinates for p in longest_skeleton_path])

In [ ]:
longest_skeleton_path = remove_skeleton_close_ends(longest_skeleton_path, closeness=20)
ends = len([p for p in longest_skeleton_path if p.end])
crosses = len([p for p in longest_skeleton_path if p.cross])

mpr_test_x = np.zeros((image.shape[0], len(longest_skeleton_path)))
mpr_test_y = np.zeros((image.shape[1], len(longest_skeleton_path)))
mpr_test_z = np.zeros((image.shape[2], len(longest_skeleton_path)))

max_index = image.shape[0]
for i, p in enumerate(longest_skeleton_path):
    coords = p.coordinates
    x_pixels = image[:, coords[1], coords[2]]
    y_pixels = image[coords[0], :, coords[2]]
    z_pixels = image[coords[0], coords[1], :]
    
    mpr_test_x[:, i] = x_pixels
    mpr_test_y[:, i] = y_pixels
    mpr_test_z[:, i] = z_pixels

_, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(mpr_test_x, cmap='gray')
axes[1].imshow(mpr_test_y, cmap='gray')
axes[2].imshow(mpr_test_z, cmap='gray')
plt.show()

In [ ]:
new_skeletons = right_skeleton + left_skeleton
new_skeleton_points = np.array([p.coordinates for p in new_skeletons])
new_skeleton_mask = np.zeros_like(skeleton_mask).astype(bool)
new_skeleton_mask[new_skeleton_points[:, 0], new_skeleton_points[:, 1], new_skeleton_points[:, 2]] = True

longest_skeleton_mask = np.zeros_like(skeleton_mask).astype(bool)
longest_skeleton_mask[longest_skeleton_path_coords[:, 0], longest_skeleton_path_coords[:, 1], longest_skeleton_path_coords[:, 2]] = True

# create_ply(image, f'4.0_image.ply')
create_ply(filtered_mask, f'4.1_filtered.ply')
create_ply(eroded_mask, f'4.2_eroded.ply')
create_ply(heartless_mask, f'4.3_heartless.ply')
create_ply(trimmed_mask, f'4.4_trimmed.ply')
create_ply(skeleton_mask, f'4.5_skeleton.ply')
create_ply(filtered_skeleton_mask, f'4.6_closest_skeletons.ply')